In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import os, gc, re, warnings
import matplotlib.pyplot as plt

import matplotlib.patches as patches
from matplotlib.patches import Arc
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

plt.style.use('fivethirtyeight')

Important Library is called for operation.

#                                WORKING WITH GAME

In [ ]:
df1 = pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')
df1

In [ ]:
df1.info()

In [ ]:
df1.describe()

In [ ]:
fig = plt.figure(figsize=(12,6))
sns.countplot(x=df1['season'], hue=df1['week'])
plt.title('Game count per Season');

This graph shows the number of game per season.

In [ ]:
fig = plt.figure(figsize=(12,6))
home = df1['homeTeamAbbr'].value_counts()
sns.barplot(x=home.index, y=home.values, ci=None)
plt.xlabel("Host Team")
plt.ylabel("Total")
plt.xticks(rotation=90);

This graph shows the data of host team.

In [ ]:
fig = plt.figure(figsize=(12,6))
visitor = df1['visitorTeamAbbr'].value_counts()
sns.barplot(x=visitor.index, y=visitor.values, ci=None)
plt.xlabel("Visitor Team")
plt.ylabel("Count")
plt.xticks(rotation=90);

# WORKING WITH PLAYERS

In [ ]:
df2=pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
df2

In [ ]:
df2.info()

In [ ]:
df2.isnull().sum()

In [ ]:
df2["birthYear"] = 0
df2["birthMonth"] = 0
df2.dropna(subset=["birthDate"], inplace=True)
for idx, row in df2.iterrows():
    if len(row['birthDate'].split('/')) == 3:  
        df2.loc[idx, 'birthYear'] = row['birthDate'].split('/')[2]
        df2.loc[idx, 'birthMonth'] = row['birthDate'].split('/')[0]
        
    elif len(row['birthDate'].split('-')) == 3:
        df2.loc[idx, 'birthYear'] = row['birthDate'].split('-')[0]
        df2.loc[idx, 'birthMonth'] = row['birthDate'].split('-')[1]

In [ ]:
df2.info()

In [ ]:
pl_height = df2["height"] 
pl_height = pl_height.apply(lambda x: x.split("-")) 
df2["height"] =pl_height.apply(lambda x: int(x[0]) * 12 + int(x[1]) if len(x) == 2 else int(x[0])) * 2.54

df2["weight"] = round(df2.weight * 0.453592, 2)

df2

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import rgb2hex
cmap = cm.get_cmap('GnBu',12) 
col_def =[]
for i in range(cmap.N):
    rgb = cmap(i)[:3]
    col_def.append(rgb2hex(rgb))
    print(rgb2hex(rgb))

In [ ]:
positon_pl = df2['Position'].value_counts()
sns.set_style('darkgrid')
fig, axes = plt.subplots(1,2,figsize=(12,6))
axes[0] = sns.barplot(x=positon_pl[:10].values, y=positon_pl[:10].index, edgecolor="purple",palette=col_def, ax=axes[0])
axes[0].set_title("Upper Ten Postions played by player (By Count)", fontsize=18)
axes[1].pie(x= positon_pl[:10], labels = positon_pl[:10].index, colors=col_def, autopct='%.0f%%',
           explode=[0.03 for i in positon_pl[:10].index])
axes[1].add_artist(plt.Circle((0,0),0.5,fc='red'))
plt.show()

In [ ]:
plt.figure(figsize=(10, 6), dpi=100)
sns.regplot(x=df2.weight, y=df2.height, line_kws={"color": "black"})
plt.title("Weight(Kg) vs Height(cm)");

In [ ]:
fig = plt.figure(figsize=(20, 15), dpi=80)

ax1 = fig.add_subplot(223)
sns.histplot(df2.weight, ax=ax1)
ax1.set_title("Weight Distribution")

ax2 = fig.add_subplot(224)
sns.histplot(df2.height, ax=ax2, bins=10)
ax2.set_title("Height Distribution");

In [ ]:
fig = plt.figure(figsize=(20, 15), dpi=80)

birthyear = df2['birthYear'].value_counts()
ax1 = fig.add_subplot(223)
sns.barplot(x=birthyear.index, y=birthyear.values, ci=None, ax=ax1)
ax1.tick_params(axis='x', rotation=45)
ax1.set_title("Birthyear Distribution",size=20)
plt.xlabel("Year", size=15)

birthmonth = df2['birthMonth'].value_counts()
ax2 = fig.add_subplot(224)
sns.barplot(x=birthmonth.index, y=birthmonth.values, ci=None, ax=ax2)
ax2.set_title("Birthmonth Distribution",size=20)
plt.xlabel("Month",size=15);

# WORKING WITH PLAYS DATAFRAME

In [ ]:
df3 = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')
df3['scoreDiff'] = abs(df3.preSnapHomeScore-df3.preSnapVisitorScore)
df3.head()

In [ ]:
fig, ((ax1,ax2),(ax3,ax4),(ax5,ax6),(ax7,ax8), (ax9,ax10)) = plt.subplots(5,2, figsize=(15,20))
df3.kickLength.plot.hist(bins=50, title='Kick length', grid=True, ax=ax1)
df3.loc[df3.kickReturnYardage.notnull()]['kickReturnYardage'].plot.hist(bins=50, title='Return result (yds)', grid=True, ax=ax2)
df3.playResult.plot.hist(bins=50, title='Play result (yds)', grid=True, ax=ax3)
df3.yardsToGo.plot.hist(bins=20, title='Yards to go at play start', grid=True, ax=ax4)
df3.penaltyYards.plot.hist(title='Penalty yards', grid=True, ax=ax5)
df3.penaltyCodes.value_counts()[:10].plot.bar(title='Penalty codes (top 10)', ax=ax6)
df3.specialTeamsPlayType.value_counts().plot.bar(title='Play type', ax=ax7)
df3.specialTeamsResult.value_counts().plot.bar(title='Play result breakdown', ax=ax8)
df3.loc[df3.passResult.notnull()]['passResult'].value_counts().plot.bar(title='Pass result breakdown', ax=ax9)
df3.yardlineNumber.plot.hist(bins=20, title='Where plays happen (yardline #)', grid=True, ax=ax10)



# SCOUTING DATASET

In [ ]:
df4 = pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
df4.head()

In [ ]:
fig, ((ax1,ax2,ax3), (ax4,ax5,ax6)) = plt.subplots(2,3, figsize=(15,8))  
df4.hangTime.plot.hist(bins=20, grid=True, title='Hangtime (seconds)', ax=ax1)
df4.loc[df4.kickType.notnull()]['kickType'].value_counts().plot.bar(title='Kick type', ax=ax2)
df4.loc[df4.kickDirectionActual.notnull()]['kickDirectionActual'].value_counts().plot.bar(title='Kick direction', ax=ax3)
df4.loc[df4.snapTime.notnull()]['snapTime'].plot.hist(bins=20, grid=True, title='Snap time', ax=ax4)
df4.loc[df4.kickContactType.notnull()]['kickContactType'].value_counts().plot.bar(title='Kick contact type', ax=ax5)
df4.loc[df4.returnDirectionActual.notnull()]['returnDirectionActual'].value_counts().plot.bar(title='Return direction', ax=ax6)

# TRACKING DATASET OF 3 YEARS

In [ ]:
df5 = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')
df6 =  pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2019.csv')
df7 =  pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2018.csv')
df5.head()
df6.head()
df7.head()

In [ ]:
print('Tracking events:')
df5.event.unique()

In [ ]:
print('Tracking events:')
df6.event.unique()

In [ ]:
print('Tracking events:')
df7.event.unique()

In [ ]:
df5['ts'] = pd.to_datetime(df5['time']).values.astype(np.int64) // 10 ** 9
df5 = df5.drop(columns=['time'])
df5.head()

In [ ]:
df6['ts'] = pd.to_datetime(df6['time']).values.astype(np.int64) // 10 ** 9
df6 = df6.drop(columns=['time'])
df6.head()

In [ ]:
df7['ts'] = pd.to_datetime(df7['time']).values.astype(np.int64) // 10 ** 9
df7 = df7.drop(columns=['time'])
df7.head()

In [ ]:
df5.groupby(['playId','nflId']).agg({'x': lambda x: x.iat[-1] - x.iat[0], 
                                       'y': lambda x: x.iat[-1] - x.iat[0], 
                                       's': 'mean',                         
                                       'dis': 'sum',                       
                                       'o': 'mean',                         
                                       'dir': 'mean',                       
                                       'frameId': 'last',                  
                                       'ts': lambda x: x.max() - x.min(), 
                                       'position': 'first', 
                                       'team': 'first',
                                       'playDirection': 'first',
                                       'event': 'first'})

In [ ]:
df6.groupby(['playId','nflId']).agg({'x': lambda x: x.iat[-1] - x.iat[0], 
                                       'y': lambda x: x.iat[-1] - x.iat[0], 
                                       's': 'mean',                         
                                       'dis': 'sum',                       
                                       'o': 'mean',                         
                                       'dir': 'mean',                       
                                       'frameId': 'last',                  
                                       'ts': lambda x: x.max() - x.min(), 
                                       'position': 'first', 
                                       'team': 'first',
                                       'playDirection': 'first',
                                       'event': 'first'})

In [ ]:
df7.groupby(['playId','nflId']).agg({'x': lambda x: x.iat[-1] - x.iat[0], 
                                       'y': lambda x: x.iat[-1] - x.iat[0], 
                                       's': 'mean',                         
                                       'dis': 'sum',                       
                                       'o': 'mean',                         
                                       'dir': 'mean',                       
                                       'frameId': 'last',                  
                                       'ts': lambda x: x.max() - x.min(), 
                                       'position': 'first', 
                                       'team': 'first',
                                       'playDirection': 'first',
                                       'event': 'first'})

In [ ]:
ball_df = pd.merge(df5.loc[df5.team=='football'], df3, how='left', on=['gameId','playId'])

fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(15,10))  
ball_df.loc[ball_df.specialTeamsPlayType=='Kickoff'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on kickoff plays (first 100 frames)', 
               ylabel='speed',
               ax=ax1)
ball_df.loc[ball_df.specialTeamsPlayType=='Punt'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on punt plays (first 100 frames)', 
               ylabel='speed',
               ax=ax2)
ball_df.loc[ball_df.specialTeamsPlayType=='Field Goal'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on field goal plays (first 100 frames)', 
               ylabel='speed',
               ax=ax3)
ball_df.loc[ball_df.specialTeamsPlayType=='Extra Point'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on extra point plays (first 100 frames)', 
               ylabel='speed',
               ax=ax4)
plt.tight_layout()

In [ ]:
ball_df = pd.merge(df6.loc[df6.team=='football'], df3, how='left', on=['gameId','playId'])

fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(15,10))  
ball_df.loc[ball_df.specialTeamsPlayType=='Kickoff'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on kickoff plays (first 100 frames)', 
               ylabel='speed',
               ax=ax1)
ball_df.loc[ball_df.specialTeamsPlayType=='Punt'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on punt plays (first 100 frames)', 
               ylabel='speed',
               ax=ax2)
ball_df.loc[ball_df.specialTeamsPlayType=='Field Goal'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on field goal plays (first 100 frames)', 
               ylabel='speed',
               ax=ax3)
ball_df.loc[ball_df.specialTeamsPlayType=='Extra Point'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on extra point plays (first 100 frames)', 
               ylabel='speed',
               ax=ax4)
plt.tight_layout()

In [ ]:
ball_df = pd.merge(df7.loc[df7.team=='football'], df3, how='left', on=['gameId','playId'])

fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2, figsize=(15,10))  
ball_df.loc[ball_df.specialTeamsPlayType=='Kickoff'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on kickoff plays (first 100 frames)', 
               ylabel='speed',
               ax=ax1)
ball_df.loc[ball_df.specialTeamsPlayType=='Punt'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on punt plays (first 100 frames)', 
               ylabel='speed',
               ax=ax2)
ball_df.loc[ball_df.specialTeamsPlayType=='Field Goal'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on field goal plays (first 100 frames)', 
               ylabel='speed',
               ax=ax3)
ball_df.loc[ball_df.specialTeamsPlayType=='Extra Point'].\
    groupby('frameId')['s'].mean()[:100]\
    .plot.line(figsize=(10,6),
               title='Avg. ball speed on extra point plays (first 100 frames)', 
               ylabel='speed',
               ax=ax4)
plt.tight_layout()

Finding Fake Player

In [ ]:
all_fakes = {'2018':'','2019':'','2020':''}
for year in all_fakes:
    print(f'Loading {year} data....')
    df = pd.read_csv(f'../input/nfl-big-data-bowl-2022/tracking{year}.csv')
    print(f'Filtering fake play data....')
    fake_play_list = df.loc[df.event.str.contains('fake')]['playId'].unique().tolist()
    all_fakes[year] = df.loc[df.playId.isin(fake_play_list)]
    print(f'Freeing memory....')
    del df
    gc.collect()
    print('Completed.')
    
fake_df = all_fakes['2018'].append(all_fakes['2019']).append(all_fakes['2020'])

print(f'\nShape of fake dataframe: {fake_df.shape}\n')
fake_df.head()

Drawing the Plot for player position

In [ ]:
plt.rcParams['figure.figsize'] = [20, 16]
def drawPitch(width, height, color="w"):
    fig = plt.figure()
    ax = plt.axes(xlim=(-10, width + 30), ylim=(-15, height + 5))
    plt.axis('off')

    # Grass around pitch
    rect = patches.Rectangle((-10, -5), width + 40, height + 10, linewidth=1, facecolor='#3f995b', capstyle='round')
    ax.add_patch(rect)


    # Pitch boundaries
    rect = plt.Rectangle((0, 0), width + 20, height, ec=color, fc="None", lw=2)
    ax.add_patch(rect)
   

    # vertical lines - every 5 yards
    for i in range(21):
        plt.plot([10 + 5 * i, 10 + 5 * i], [0, height], c="w", lw=2)
    
        
    # distance markers - every 10 yards
    for yards in range(10, width, 10):
        yards_text = yards if yards <= width / 2 else width - yards
        # top markers
        plt.text(10 + yards - 2, height - 7.5, yards_text, size=20, c="w", weight="bold")
        # botoom markers
        plt.text(10 + yards - 2, 7.5, yards_text, size=20, c="w", weight="bold", rotation=180)
   

    # yards markers - every yard
    # bottom markers
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [1, 3], color="w", lw=3)

    # top markers
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [height - 1, height - 3], color="w", lw=3)

    # middle bottom markers
    y = (height - 18.5) / 2
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [y, y + 2], color="w", lw=3)

    # middle top markers
    for x in range(20):
        for j in range(1, 5):
            plt.plot([10 + x * 5 + j, 10 + x * 5 + j], [height - y, height - y - 2], color="w", lw=3)
   

    # draw home end zone
    plt.text(2.5, (height - 10) / 2, "HOME", size=40, c="w", weight="bold", rotation=90)
    rect = plt.Rectangle((0, 0), 10, height, ec=color, fc="#0064dc", lw=2)
    ax.add_patch(rect)

    # draw away end zone    
    plt.text(112.5, (height - 10) / 2, "AWAY", size=40, c="w", weight="bold", rotation=-90)
    rect = plt.Rectangle((width + 10, 0), 10, height, ec=color, fc="#c80014", lw=2)
    ax.add_patch(rect)
    
    # draw extra spot point
    # left
    y = (height - 3) / 2
    plt.plot([10 + 2, 10 + 2], [y, y + 3], c="w", lw=2)
    
    # right
    plt.plot([width + 10 - 2, width + 10 - 2], [y, y + 3], c="w", lw=2)
    
    # draw goalpost
    goal_width = 6 # yards
    y = (height - goal_width) / 2
    # left
    plt.plot([0, 0], [y, y + goal_width], "-", c="y", lw=10, ms=20)
    # right
    plt.plot([width + 20, width + 20], [y, y + goal_width], "-", c="y", lw=10, ms=20)
    
    return fig, ax

fig, ax = drawPitch(100, 53.3)

Conclusion -
Finally it has completed and I also took help from different source . It was a great project!